## Bag of Words(BoW)

In [1]:
from konlpy.tag import Okt
okt=Okt()  

token="내꺼인듯 내꺼아닌 내꺼같은 너란 사람은 참"
token=okt.morphs(token) # 형태소분석
word2index={}  
bow=[]
# 토큰하나씩 들고온다
for voca in token:
    # word2index 안에 해당 토큰이 없다면
    if voca not in word2index:  
        # 인덱스(word2index길이만큼)를 지정해주고
        word2index[voca]=len(word2index)
        # 단어가 등장했으니, 빈도수를 담는 리스트에 1을 넣어준다.
        bow.insert(len(word2index)-1,1)
    # word2index 안에 있는 토큰이라면
    else:
        # 해당 토큰의 인덱스를 가져와서
        index=word2index[voca]
        # 한 번 더나왔으니 그 토큰의 인덱스로 빈도수 리스트에 접근하여 값을 1을 더해준다
        bow[index]=bow[index]+1
print(word2index) # 단어에 인덱스 부여
print(bow)        # 빈도수저장


{'내꺼인듯': 0, '내꺼': 1, '아닌': 2, '같은': 3, '너': 4, '란': 5, '사람': 6, '은': 7, '참': 8}
[1, 2, 1, 1, 1, 1, 1, 1, 1]


In [2]:
# scikit-learn 이용하기
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    '내꺼인듯 내꺼아닌 내꺼같은 너란 사람은 참',
    '아니 왜 한 글 자 는 없 어 지 는 거 지'
]
vector = CountVectorizer()

print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다. 

[[1 1 1 1 1 0]
 [0 0 0 0 0 1]]
{'내꺼인듯': 2, '내꺼아닌': 1, '내꺼같은': 0, '너란': 3, '사람은': 4, '아니': 5}


## TF-IDF

In [3]:
import pandas as pd
from math import log

In [4]:
docs=[
    ['안녕', '나는', '자연어는', '연어가', '아니야'],
    ['연어' ,'먹고싶은' ,'연어', '덕후'],
    ['안녕' ,'덕후' ,'연어' ,'먹으러' ,'가자'],
    ['자연어' ,'처리' ,'어려워']
]

In [5]:
N=len(docs)
# 문서내 단어 빈도수
def tf(word,doc):
    return doc.count(word)

# 해당 단어 문서내 등장 유무
def idf(word):
    df=0
    for doc in docs:
        df += word in doc
    return log(N / (1+df))

def tfidf(word, doc):
    return tf(word,doc) * idf(word)

In [6]:
# vocab 만들기

vocab_=defaultdict(int)
for doc in docs:
    for word in doc:
        vocab_[word]+=1
print(vocab_)
vocab__=sorted(vocab_.items(),key=lambda x:x[1])
vocab=[i[0] for i in vocab__]
print(vocab)

defaultdict(<class 'int'>, {'안녕': 2, '나는': 1, '자연어는': 1, '연어가': 1, '아니야': 1, '연어': 3, '먹고싶은': 1, '덕후': 2, '먹으러': 1, '가자': 1, '자연어': 1, '처리': 1, '어려워': 1})
['나는', '자연어는', '연어가', '아니야', '먹고싶은', '먹으러', '가자', '자연어', '처리', '어려워', '안녕', '덕후', '연어']


In [7]:
# DTM 만들기
dtm=[]
for doc in docs:
    tmp_list=[]
    for word in vocab:
        tf_=tf(word,doc)
        tmp_list.append(tf_)
    dtm.append(tmp_list)
df=pd.DataFrame(dtm,columns=vocab)
df

,나는,자연어는,연어가,아니야,먹고싶은,먹으러,가자,자연어,처리,어려워,안녕,덕후,연어
0,1,1,1,1,0,0,0,0,0,0,1,0,0
1,0,0,0,0,1,0,0,0,0,0,0,1,2
2,0,0,0,0,0,1,1,0,0,0,1,1,1
3,0,0,0,0,0,0,0,1,1,1,0,0,0


In [8]:
# TF-IDF matrix 만들기
tfidf_=[]
for doc in docs:
    tmp_list=[]
    for word in vocab:
        tmp_list.append(tfidf(word,doc))
    tfidf_.append(tmp_list)
df_result=pd.DataFrame(tfidf_,columns=vocab)
df_result


,나는,자연어는,연어가,아니야,먹고싶은,먹으러,가자,자연어,처리,어려워,안녕,덕후,연어
0,0.693147,0.693147,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.287682,0.575364
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,0.000000,0.000000,0.000000,0.287682,0.287682,0.287682
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,0.693147,0.000000,0.000000,0.000000
